In [ ]:
import os

from getpass import getpass
import warnings
warnings.filterwarnings('ignore')
from utils import ChatOpenAI

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

In [24]:
from langchain.agents import tool, load_tools
import math
import pandas as pd
from tqdm import tqdm
from langchain.agents import initialize_agent, AgentType
from langchain.chains import LLMMathChain
import warnings
warnings.filterwarnings('ignore')


In [ ]:

# _______________________________________________ЗАДАНИЕ 3.3.8________________________________________________________________

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110884/custom_math_tools.csv")
df

In [ ]:


@tool
def compute_arc_length(radius, angle_in_degrees)-> int:
    """Вычисляет длину дуги по радиусу и углу в градусах"""
    return (angle_in_degrees / 360) * 2 * math.pi * radius


@tool
def compute_triangle_area(a, b, c)-> int:
    """Вычисляет площадь треугольника по длинам его сторон"""
    s = (a + b + c) / 2
    return math.sqrt(s * (s - a) * (s - b) * (s - c))

# Реализуйте остальные функции
@tool
def add_numbers(num1, num2)-> int:
    """Складывает два числа."""
    return num1 + num2
@tool
def convert_meters_to_cm(num)-> int:
    """Переводит метры в сантиметры."""
    return num * 100
@tool
def convert_cubic_cm_to_liters(num)-> int:
    """Переводит кубические сантиметры в литры."""
    return num / 1000
@tool
def compute_rectangle_perimeter(a, b)-> int:
    """Вычисляет периметр прямоугольника."""
    return 2 * (a + b)
@tool
def compute_circle_area(radius)-> int:
    """Вычисляет площадь круга."""
    return math.pi * (radius ** 2)
@tool
def compute_cylinder_volume(radius, height)-> int:
    """Вычисляет объем цилиндра."""
    return math.pi * radius ** 2 * height
@tool
def compute_cube_volume(a)-> int:
    """Вычисляет объем куба."""
    return a ** 3
@tool
def convert_binary_to_decimal(binary_number: str)-> int:
    """Переводит число из двоичной системы счисления в десятичную систему счисления."""
    return int(binary_number, 2)
@tool
def convert_decimal_to_binary(decimal_number)-> int:
    """Переводит число из десятичной системы счисления в двоичную систему счисления."""
    return bin(decimal_number)[2:]
@tool
def get_count_ones(number: str)-> int:
    """Находит количество единиц в двоичном представлении числа."""
    return number.count('1')
                        
# Поместите все написанные функции в tools      
tools = [compute_arc_length, compute_triangle_area,add_numbers,convert_meters_to_cm, convert_cubic_cm_to_liters,compute_rectangle_perimeter, compute_circle_area,compute_cylinder_volume,compute_cube_volume,convert_binary_to_decimal,convert_decimal_to_binary,get_count_ones]

In [ ]:

agent = initialize_agent(
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools, llm=llm, 
    verbose=True, 
    handle_parsing_errors=True,
    max_iterations=10
    )



In [ ]:
answers = []
for task in df['task']:

    try:

        # Попытка обработки задачи агентом

        result = agent.run(f'{task}. Ответ должен быть представлен в виде целого числа, если итоговый результат является точным целым числом, или в виде вещественного числа с точностью до двух знаков после запятой, если результат содержит дробную часть. Не допускаеться в ответе содержания слов')

        print(f"Задача: {task}\nРезультат: {result}\n")  # Выводим успешный результат

    except Exception as e:

        # Сохранение ошибки

        result = f"Ошибка: {e}"

        print(f"Задача: {task}\nРезультат: {result}\n")  # Выводим ошибку

    answers.append(result)

 

# Добавление столбца с ответами

df['answer'] = answers
# Сохранение в CSV

df[['task', 'answer']].to_csv("3.3.8_solution.csv", index=False)